In [1]:
# elasticsearch에 넣고 싶은 데이터 읽어오기

import pandas as pd
data=pd.read_csv('Report.csv')
data

,자치구,세대,총인구,남자,여자
0,합계,4421143,9765869,4739972,5025897
1,종로구,73652,154318,74561,79757
2,중구,63267,131943,64274,67669
3,용산구,111177,238300,115655,122645
4,성동구,134620,294140,143055,151085
5,광진구,167438,353380,169739,183641
6,동대문구,167591,351626,172463,179163
7,중랑구,185497,393149,193652,199497
8,성북구,195099,441717,212069,229648
9,강북구,144573,304257,147881,156376


In [2]:
# elasticsearch를 python에 연동하기

from elasticsearch import Elasticsearch
import requests
import csv

directory_path = 'path'
res = requests.get('http://localhost:9200')
es = Elasticsearch([{'host':'localhost','port':'9200'}])

In [ ]:
# mapping이 필요하면 mapping하기
# 지금은 mapping이 필요없어서 실행하지 않는다.
# 참고로 elasticsearch는 mapping하는 능력이 매우 뛰어나다.

# es.indices.create(
#     index='dictionary3',
#     body={
#         "mappings": {
#             "properties": {
#                 "location": {
#                     "type": "geo_point",
#                 }
#                 "lon": {
#                     "type": "geo_point",
#                 },
#                 "lat": {
#                     "type": "geo_point",
#                    }
#                 }
#             }
#         }
# )


In [3]:
# elasticsearch에 알맞는 쿼리로 데이터 집어넣기
# for i in range(len(data['세대']): 를 사용하지 않으면 하나의 _doc에 모든 값이 들어간다.

for i in range(len(data['자치구'])):
    es.index(index="tableau2", 
        doc_type='_doc', 
        document={'자치구':data['자치구'][i],      # body는 업그레이드 버전에서 사라질 예정이며 body대신 document를 사용해라
        '세대':data['세대'][i],
        '총인구':data['총인구'][i], 
        '남자':data['남자'][i],
        '여자':data['여자'][i]
                 })


D:\Anaconda\envs\mongta\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
# elasticsearch에 알맞는 쿼리로 데이터 집어넣기
# for i in range(len(data['세대']): 를 사용하지 않으면 하나의 _doc에 모든 값이 들어간다.
# 하루에 한 번씩 크롤링해서 넣어야되는 데이터의 경우 
# index_name = 'kmh_test4'+"_"+today 과
# es.indices.put_alias(index = index_name, name = 'kmh_test4') 을 사용한다.
# elasticsearch에 보면 'tableau2' 가 가장 상위의 index로 생성이 되고 
# 'tableau2_2021.12.10','tableau2_2021.12.11' 등이 하위의 index로 생성이 될것이다.

# from datetime import datetime

# now = datetime.now()
# today = now.strftime('%Y-%m-%d')

# index_name = 'tableau2'+"_"+today

# for i in range(len(info['url'])): 
#         es.index(index=index_name, 
#             doc_type='_doc', 
#             document={'title':info['title'][i],         # body는 업그레이드 버전에서 사라질 예정이며 body대신 document를 사용해라.
#             'titlekeyword':info['titlekeyword'][i],
#             'url':info['url'][i], 
#             'company':info['company'][i],
#             'contents':info['contents'][i], 
#             'date':info['date'][i],
#             'image':info['image'][i],
#             'keyword':info['keyword'][i],
#             'category':info['category'][i]})

# es.indices.put_alias(index = index_name, name = 'kmh_test4')

### 오류가 생긴다면 `같은 index`에 값을 넣지 말자. 기존의 데이터가 덮여씌우기 때문에 오류가 발생한다.
##### 필자도 이래서 오류가 생겼다.. (무려 1시간 헤멤....)